In [2]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [3]:
#import exp_filtered
data = pd.read_csv('./experiments_filtered.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.head(10)


,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
2,390,2023-10-04 18:52:03.080773000,2023-10-04 19:22:03.086651800,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800,1696445973,1696446873
3,391,2023-10-04 19:25:37.153339900,2023-10-04 19:55:37.155585600,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 19:33:07.153339900,2023-10-04 19:48:07.155585600,1696447987,1696448887
6,505,2023-11-07 18:12:32.614239000,2023-11-07 18:42:32.616707000,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-11-07 18:20:02.614239000,2023-11-07 18:35:02.616707000,1699381202,1699382102
7,506,2023-11-07 18:46:01.613382700,2023-11-07 19:16:01.615839300,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-11-07 18:53:31.613382700,2023-11-07 19:08:31.615839300,1699383211,1699384111
8,393,2023-10-04 20:32:47.309861600,2023-10-04 21:02:47.314917800,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 20:40:17.309861600,2023-10-04 20:55:17.314917800,1696452017,1696452917
9,394,2023-10-04 21:06:22.775563400,2023-10-04 21:36:22.778880000,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 21:13:52.775563400,2023-10-04 21:28:52.778880000,1696454032,1696454932
11,507,2023-11-07 19:19:31.901645600,2023-11-07 19:49:31.904968200,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-11-07 19:27:01.901645600,2023-11-07 19:42:01.904968200,1699385221,1699386121
14,396,2023-10-04 22:13:33.814670400,2023-10-04 22:43:33.817409400,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:21:03.814670400,2023-10-04 22:36:03.817409400,1696458063,1696458963
15,397,2023-10-04 22:47:09.383711000,2023-10-04 23:17:09.386844200,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:54:39.383711000,2023-10-04 23:09:39.386844200,1696460079,1696460979
17,510,2023-11-07 21:00:03.559839700,2023-11-07 21:30:03.562290200,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-11-07 21:07:33.559839700,2023-11-07 21:22:33.562290200,1699391253,1699392153


In [4]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,identifier,experiment,type
0,2023-11-22 03:16:59.349471+00:00,16,663,16.0
0,2023-11-22 03:16:59.346725+00:00,16,663,17.0
1,2023-11-22 03:16:59.349417+00:00,16,663,17.0
0,2023-11-22 03:09:48.319101+00:00,16,663,3.0
1,2023-11-22 03:09:48.320160+00:00,16,663,3.0
2,2023-11-22 03:09:48.320172+00:00,16,663,2.0
3,2023-11-22 03:09:48.320372+00:00,16,663,3.0
4,2023-11-22 03:09:48.320454+00:00,16,663,2.0
5,2023-11-22 03:09:48.320489+00:00,16,663,2.0
6,2023-11-22 03:09:48.320499+00:00,16,663,2.0


In [5]:
# Transpose the table -> type and count
df = messages.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='1s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="1s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_115830/3621685534.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:04+00:00,3748.0,520.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,478.0,59.0,0.0,0.0,0.0,347.0
2,390,2023-10-04 18:52:07+00:00,3986.0,542.0,0.0,0.0,0.0,0.0
3,390,2023-10-04 18:52:08+00:00,472.0,72.0,0.0,0.0,0.0,351.0
4,390,2023-10-04 18:52:10+00:00,3703.0,503.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:53:47+00:00,3542.0,481.0,0.0,0.0,0.0,321.0
96,390,2023-10-04 18:53:48+00:00,1434.0,202.0,0.0,0.0,0.0,0.0
97,390,2023-10-04 18:53:49+00:00,134.0,18.0,0.0,0.0,0.0,0.0
98,390,2023-10-04 18:53:50+00:00,2485.0,338.0,0.0,0.0,0.0,322.0


In [6]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

eventsTS.to_csv('./events.csv')
eventsTS.head(500)

,experiment,_time,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:04+00:00,3748.0,520.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,478.0,59.0,0.0,0.0,0.0,347.0
2,390,2023-10-04 18:52:07+00:00,3986.0,542.0,0.0,0.0,0.0,0.0
3,390,2023-10-04 18:52:08+00:00,472.0,72.0,0.0,0.0,0.0,351.0
4,390,2023-10-04 18:52:10+00:00,3703.0,503.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
495,390,2023-10-04 19:00:38+00:00,1482.0,228.0,0.0,0.0,0.0,181.0
496,390,2023-10-04 19:00:39+00:00,351.0,59.0,0.0,0.0,0.0,0.0
497,390,2023-10-04 19:00:40+00:00,152.0,21.0,0.0,0.0,0.0,0.0
498,390,2023-10-04 19:00:41+00:00,1498.0,227.0,0.0,0.0,0.0,180.0


In [7]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess2_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)

    data = pd.read_csv('../../datasets/partialTables/pub_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_pub = pd.DataFrame(data)
    
    deliverMessage = pd.concat([query_mess, deliverMessage])
    publishMessage = pd.concat([publishMessage, query_pub])

    del query_pub
    del query_mess

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2023-11-22 03:09:48.319101+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
1,2023-11-22 03:09:48.320160+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
2,2023-11-22 03:09:48.320372+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
3,2023-11-22 03:09:48.320578+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
4,2023-11-22 03:09:48.320690+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
5,2023-11-22 03:09:48.320711+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
6,2023-11-22 03:09:48.320974+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
7,2023-11-22 03:09:48.321057+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
8,2023-11-22 03:09:48.321534+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663
9,2023-11-22 03:09:48.322297+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,663


In [8]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134903+00:00,753
1,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136298+00:00,2148
2,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136886+00:00,2736
3,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137090+00:00,2940
4,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137205+00:00,3055
5,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137276+00:00,3126
6,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137353+00:00,3203
7,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137377+00:00,3227
8,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134975+00:00,308
9,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.135169+00:00,502


In [9]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='1s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_115830/3819993095.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:04+00:00,1856.745902
1,390,2023-10-04 18:52:05+00:00,1979.913793
2,390,2023-10-04 18:52:07+00:00,1670.648148
3,390,2023-10-04 18:52:08+00:00,1710.708333
4,390,2023-10-04 18:52:10+00:00,1879.665217
...,...,...,...
95,390,2023-10-04 18:53:50+00:00,2067.397163
96,390,2023-10-04 18:53:51+00:00,1486.214286
97,390,2023-10-04 18:53:53+00:00,1588.239130
98,390,2023-10-04 18:53:54+00:00,1466.568000


In [10]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:04+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,1979.913793,478.0,59.0,0.0,0.0,0.0,347.0
2,390,2023-10-04 18:52:07+00:00,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0
3,390,2023-10-04 18:52:08+00:00,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0
4,390,2023-10-04 18:52:10+00:00,1879.665217,3703.0,503.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:53:50+00:00,2067.397163,2485.0,338.0,0.0,0.0,0.0,322.0
96,390,2023-10-04 18:53:51+00:00,1486.214286,1439.0,203.0,0.0,0.0,0.0,0.0
97,390,2023-10-04 18:53:53+00:00,1588.239130,2498.0,335.0,0.0,0.0,0.0,320.0
98,390,2023-10-04 18:53:54+00:00,1466.568000,1449.0,202.0,0.0,0.0,0.0,0.0


In [11]:
prop.to_csv('./propagationTime.csv')

In [12]:
#Bandwidth

#Load rpc dataset
# data = pd.read_csv('./Extraction/rpc.csv',header=0, index_col=0)
# rpc = pd.DataFrame(data)
# rpc = rpc.rename(columns={'rpc':'count'})

# rpc.head(10)

rpc = pd.DataFrame()


for idx, exec in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/rpc_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.134925+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.135840+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.135852+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.135928+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.135930+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.135938+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.135962+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.135988+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.136030+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [13]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='1s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_115830/682052193.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [14]:
rpc.head(10)

,experiment,_time,rpc
0,390,2023-10-04 18:52:04+00:00,4243
1,390,2023-10-04 18:52:05+00:00,697
2,390,2023-10-04 18:52:07+00:00,4494
3,390,2023-10-04 18:52:08+00:00,715
4,390,2023-10-04 18:52:10+00:00,4161
5,390,2023-10-04 18:52:11+00:00,774
6,390,2023-10-04 18:52:13+00:00,3879
7,390,2023-10-04 18:52:14+00:00,1055
8,390,2023-10-04 18:52:16+00:00,3913
9,390,2023-10-04 18:52:17+00:00,1053


In [15]:
#Total bandwidth

# data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
# message = pd.DataFrame(data)

# idx = [3.0]
# message = message[message["type"].isin(idx)]
# message = message.drop(columns=["type", "identifier"])

message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='1s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_115830/1755436038.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:04+00:00,521
1,390,2023-10-04 18:52:05+00:00,59
2,390,2023-10-04 18:52:07+00:00,547
3,390,2023-10-04 18:52:08+00:00,72
4,390,2023-10-04 18:52:10+00:00,505
5,390,2023-10-04 18:52:11+00:00,73
6,390,2023-10-04 18:52:13+00:00,465
7,390,2023-10-04 18:52:14+00:00,115
8,390,2023-10-04 18:52:16+00:00,464
9,390,2023-10-04 18:52:17+00:00,115


In [16]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='1s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_115830/3938913423.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:04+00:00,4764
1,390,2023-10-04 18:52:05+00:00,756
2,390,2023-10-04 18:52:07+00:00,5041
3,390,2023-10-04 18:52:08+00:00,787
4,390,2023-10-04 18:52:10+00:00,4666
...,...,...,...
95,390,2023-10-04 18:53:47+00:00,4650
96,390,2023-10-04 18:53:48+00:00,1818
97,390,2023-10-04 18:53:49+00:00,173
98,390,2023-10-04 18:53:50+00:00,3293


In [17]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:04+00:00,4764
1,390,2023-10-04 18:52:05+00:00,756
2,390,2023-10-04 18:52:07+00:00,5041
3,390,2023-10-04 18:52:08+00:00,787
4,390,2023-10-04 18:52:10+00:00,4666
5,390,2023-10-04 18:52:11+00:00,847
6,390,2023-10-04 18:52:13+00:00,4344
7,390,2023-10-04 18:52:14+00:00,1170
8,390,2023-10-04 18:52:16+00:00,4377
9,390,2023-10-04 18:52:17+00:00,1168


In [18]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipBandwidth
0,390,2023-10-04 18:52:04+00:00,4243
1,390,2023-10-04 18:52:05+00:00,697
2,390,2023-10-04 18:52:07+00:00,4494
3,390,2023-10-04 18:52:08+00:00,715
4,390,2023-10-04 18:52:10+00:00,4161
...,...,...,...
95,390,2023-10-04 18:53:47+00:00,4167
96,390,2023-10-04 18:53:48+00:00,1615
97,390,2023-10-04 18:53:49+00:00,155
98,390,2023-10-04 18:53:50+00:00,2955


In [19]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:04+00:00,521
1,390,2023-10-04 18:52:05+00:00,59
2,390,2023-10-04 18:52:07+00:00,547
3,390,2023-10-04 18:52:08+00:00,72
4,390,2023-10-04 18:52:10+00:00,505
5,390,2023-10-04 18:52:11+00:00,73
6,390,2023-10-04 18:52:13+00:00,465
7,390,2023-10-04 18:52:14+00:00,115
8,390,2023-10-04 18:52:16+00:00,464
9,390,2023-10-04 18:52:17+00:00,115


In [20]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_115830/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_115830/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 18:52:04+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,521.0
1,390,2023-10-04 18:52:05+00:00,1979.913793,478.0,59.0,0.0,0.0,0.0,347.0,59.0
2,390,2023-10-04 18:52:07+00:00,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0,547.0
3,390,2023-10-04 18:52:08+00:00,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0,72.0
4,390,2023-10-04 18:52:10+00:00,1879.665217,3703.0,503.0,0.0,0.0,0.0,0.0,505.0
...,...,...,...,...,...,...,...,...,...,...
495,390,2023-10-04 19:02:14+00:00,1727.687500,789.0,116.0,0.0,0.0,0.0,145.0,116.0
496,390,2023-10-04 19:02:15+00:00,1359.677966,872.0,136.0,0.0,0.0,0.0,0.0,136.0
497,390,2023-10-04 19:02:16+00:00,1691.687500,916.0,132.0,0.0,0.0,0.0,0.0,133.0
498,390,2023-10-04 19:02:17+00:00,1864.187500,473.0,75.0,0.0,0.0,0.0,146.0,75.0


In [21]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = totalBand['totalBandwidth']+totalBand['messageDuplicated']

totalBand.head(500)

/tmp/ipykernel_115830/3458332421.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_115830/3458332421.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 18:52:04+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,521.0,8512.0
1,390,2023-10-04 18:52:05+00:00,1979.913793,478.0,59.0,0.0,0.0,0.0,347.0,59.0,1234.0
2,390,2023-10-04 18:52:07+00:00,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0,547.0,9027.0
3,390,2023-10-04 18:52:08+00:00,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0,72.0,1259.0
4,390,2023-10-04 18:52:10+00:00,1879.665217,3703.0,503.0,0.0,0.0,0.0,0.0,505.0,8369.0
...,...,...,...,...,...,...,...,...,...,...,...
495,390,2023-10-04 19:02:14+00:00,1727.687500,789.0,116.0,0.0,0.0,0.0,145.0,116.0,1873.0
496,390,2023-10-04 19:02:15+00:00,1359.677966,872.0,136.0,0.0,0.0,0.0,0.0,136.0,2000.0
497,390,2023-10-04 19:02:16+00:00,1691.687500,916.0,132.0,0.0,0.0,0.0,0.0,133.0,2084.0
498,390,2023-10-04 19:02:17+00:00,1864.187500,473.0,75.0,0.0,0.0,0.0,146.0,75.0,1153.0


In [22]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_115830/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_115830/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth
0,390,2023-10-04 18:52:04+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,521.0,8512.0,4243.0
1,390,2023-10-04 18:52:05+00:00,1979.913793,478.0,59.0,0.0,0.0,0.0,347.0,59.0,1234.0,697.0
2,390,2023-10-04 18:52:07+00:00,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0,547.0,9027.0,4494.0
3,390,2023-10-04 18:52:08+00:00,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0,72.0,1259.0,715.0
4,390,2023-10-04 18:52:10+00:00,1879.665217,3703.0,503.0,0.0,0.0,0.0,0.0,505.0,8369.0,4161.0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,390,2023-10-04 19:02:14+00:00,1727.687500,789.0,116.0,0.0,0.0,0.0,145.0,116.0,1873.0,968.0
496,390,2023-10-04 19:02:15+00:00,1359.677966,872.0,136.0,0.0,0.0,0.0,0.0,136.0,2000.0,992.0
497,390,2023-10-04 19:02:16+00:00,1691.687500,916.0,132.0,0.0,0.0,0.0,0.0,133.0,2084.0,1035.0
498,390,2023-10-04 19:02:17+00:00,1864.187500,473.0,75.0,0.0,0.0,0.0,146.0,75.0,1153.0,605.0


In [23]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.to_csv('./consDiscovery.csv')

consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth
0,390,2023-10-04 18:52:04+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,521.0,8512.0,4243.0
1,390,2023-10-04 18:52:05+00:00,1979.913793,478.0,59.0,0.0,0.0,0.0,347.0,59.0,1234.0,697.0
2,390,2023-10-04 18:52:07+00:00,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0,547.0,9027.0,4494.0
3,390,2023-10-04 18:52:08+00:00,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0,72.0,1259.0,715.0
4,390,2023-10-04 18:52:10+00:00,1879.665217,3703.0,503.0,0.0,0.0,0.0,0.0,505.0,8369.0,4161.0
5,390,2023-10-04 18:52:11+00:00,1723.305556,538.0,73.0,0.0,0.0,0.0,347.0,73.0,1385.0,774.0
6,390,2023-10-04 18:52:13+00:00,1816.257895,3430.0,465.0,0.0,0.0,0.0,0.0,465.0,7774.0,3879.0
7,390,2023-10-04 18:52:14+00:00,1556.234783,786.0,115.0,0.0,0.0,0.0,342.0,115.0,1956.0,1055.0
8,390,2023-10-04 18:52:16+00:00,2597.587302,3456.0,460.0,0.0,0.0,0.0,0.0,464.0,7833.0,3913.0
9,390,2023-10-04 18:52:17+00:00,1654.754386,779.0,115.0,0.0,0.0,0.0,343.0,115.0,1947.0,1053.0


In [24]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter', 'originalStart', 'originalEnd']) 

consFinal.to_csv('./consolidated.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,521.0,8512.0,4243.0,unl,8,6,12,4,8,2,0.25,1.0
1,390,1979.913793,478.0,59.0,0.0,0.0,0.0,347.0,59.0,1234.0,697.0,unl,8,6,12,4,8,2,0.25,1.0
2,390,1670.648148,3986.0,542.0,0.0,0.0,0.0,0.0,547.0,9027.0,4494.0,unl,8,6,12,4,8,2,0.25,1.0
3,390,1710.708333,472.0,72.0,0.0,0.0,0.0,351.0,72.0,1259.0,715.0,unl,8,6,12,4,8,2,0.25,1.0
4,390,1879.665217,3703.0,503.0,0.0,0.0,0.0,0.0,505.0,8369.0,4161.0,unl,8,6,12,4,8,2,0.25,1.0
5,390,1723.305556,538.0,73.0,0.0,0.0,0.0,347.0,73.0,1385.0,774.0,unl,8,6,12,4,8,2,0.25,1.0
6,390,1816.257895,3430.0,465.0,0.0,0.0,0.0,0.0,465.0,7774.0,3879.0,unl,8,6,12,4,8,2,0.25,1.0
7,390,1556.234783,786.0,115.0,0.0,0.0,0.0,342.0,115.0,1956.0,1055.0,unl,8,6,12,4,8,2,0.25,1.0
8,390,2597.587302,3456.0,460.0,0.0,0.0,0.0,0.0,464.0,7833.0,3913.0,unl,8,6,12,4,8,2,0.25,1.0
9,390,1654.754386,779.0,115.0,0.0,0.0,0.0,343.0,115.0,1947.0,1053.0,unl,8,6,12,4,8,2,0.25,1.0
